In [54]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import quad
from scipy.optimize import minimize

np.random.seed(0)

# Exercise 1

In [107]:
c   = 3e5
mpc = 1

def modular_dist(z, h0, omega_l, omega_k, omega_m, omega_r):

    norm_hubble_inv = lambda x: 1/np.sqrt(omega_l + omega_k*(1+x)**2 + omega_m*(1+x)**3 + omega_r*(1+x)**4)
    comoving_dist = quad(norm_hubble_inv, 0, z)[0]
    if omega_k == 0:
        temp_comoving_dist = comoving_dist
    elif omega_k > 0:
        temp_comoving_dist = np.sinh(np.sqrt(omega_k)*comoving_dist)/np.sqrt(omega_k)
    else:
        temp_comoving_dist = np.sin(np.sqrt(-omega_k)*comoving_dist)/np.sqrt(-omega_k)
    lumin_dist = (1+z)*temp_comoving_dist
    mod_dist = 5*np.log10(lumin_dist) + 25 + 5*np.log10(c/h0/mpc)

    return mod_dist

def log_likelihood(z_data, modular_dist_data, sigma_data, h0, omega_l, omega_k, omega_m, omega_r):

    likelihood = 0

    for z, mu, sigma in zip(z_data, modular_dist_data, sigma_data):
        likelihood += (modular_dist(z, h0, omega_l, omega_k, omega_m, omega_r) - mu)**2/sigma**2

    return likelihood


sn_data = open("nc_data_snia_diag_legacy.obj", "r")
lines   = sn_data.readlines()

for line in lines:
    
    obj = line.split(':')
    key = obj[0].strip("' ,\n")

    if key == 'mean':
        data = obj[1].strip(" <>,\n[]").split(', ')
        dist_obs = [float(d) for d in data]
    elif key == 'sigma':
        data = obj[1].strip(" <>,\n[]").split(', ')
        sigma_obs = [float(d) for d in data]
    elif key == 'z':
        data = obj[1].strip(" <>,\n[]").split(', ')
        z_obs = [float(d) for d in data]

## a)

Calculing best fit

In [143]:
fixed_likelihood = lambda theta: log_likelihood(z_obs, dist_obs, sigma_obs, theta[0], theta[1], 0, theta[2], 0)

guess = [70,0.7,0.3]
best_fit = minimize(fixed_likelihood, guess, method='nelder-mead', tol=1e-20)

In [144]:
best_fit.x

array([67.71237825,  0.78967993,  0.28097493])

## b)

In [158]:
free_likelihood = lambda theta: log_likelihood(z_obs, dist_obs, sigma_obs, theta[0], theta[1], theta[2], theta[3], theta[4])

guess = [70, 0.7, 0, 0.3, 0]
best_fit = minimize(free_likelihood, guess, method='nelder-mead', tol=1e-20)

In [159]:
best_fit.x

array([ 5.53840271e+01,  1.18894033e+00, -2.94849592e-03,  4.05059987e-01,
        9.72450554e-03])